In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Button, Slider
from scipy import integrate
from scipy.fft import rfft
from functools import partial

## Q6: Damped driven pendulum and chaos

There are a large class of ODE integration methods available through the `scipy.integrate.ode()` function.  Not all of them provide _dense output_ -- most will just give you the value at the end of the integration.  

The explicit Runge-Kutta integrator will give you access to the solution at intermediate points and provides methods to interpolate to any value.  You enable this via `dense_output=True`.

The damped driven pendulum obeys the following equations:

$$\dot{\theta} = \omega$$

$$\dot{\omega} = -q \omega - \sin \theta + b \cos \omega_d t$$

here, $\theta$ is the angle of the pendulum from vertical and $\omega$ is the angular velocity.  $q$ is a damping coefficient, $b$ is a forcing amplitude, and $\omega_d$ is a driving frequency.

Choose $q = 0.5$ and $\omega_d = 2/3$.

Integrate the system for different values of $b$ (start with $b = 0.9$ and increase by $0.05$, and plot the results ($\theta$ vs. $t$).  Here's a RHS function to get you started:

In [ ]:
def rhs(t, Y, q, omega_d, b):
        """ damped driven pendulum system derivatives.  Here, Y = (theta, omega) are
        the solution variables. """
        f = np.zeros_like(Y)

        f[0] = Y[1]
        f[1] = -q*Y[1] - np.sin(Y[0]) + b*np.cos(omega_d*t)

        return f

Note that the pendulum can flip over, giving values of $\theta$ outside of $[-\pi, \pi]$.  The following function can be used to restrict it back to $[-\pi, \pi]$ for plotting.

In [ ]:
def restrict_theta(theta):
    """ convert theta to be restricted to lie between -pi and pi"""
    tnew = theta + np.pi
    tnew += -2.0*np.pi*np.floor(tnew/(2.0*np.pi))
    tnew -= np.pi
    return tnew

Write a function that takes an initial angle, $\theta_0$, and integrates the system and returns the solution.

Note, the righthand side function, `rhs`, takes additional arguments that you need to pass through the integrator.  The preferred method to do this with the `solve_ivp()` interface appears to be to use `functools.partial()`, as:
```
from functools import partial

r = solve_ivp(partial(rhs, q=q, omega_d=omega_d, b=b), ...)
```

Some values of $b$ will show very non-periodic behavior.  To see chaos, integrate two different pendula that are the same except for $\theta_0$, with only a small difference between then (like 60 degrees and 60.0001 degrees.  You'll see the solutions track for a while, but then diverge.

In [ ]:
def restrict_theta(theta):
    """ convert theta to be restricted to lie between -pi and pi"""
    tnew = theta + np.pi
    tnew += -2.0*np.pi*np.floor(tnew/(2.0*np.pi))
    tnew -= np.pi
    return tnew

def rhs(t, Y, q, omega_d, b):
        """ damped driven pendulum system derivatives.  Here, Y = (theta, omega) are
        the solution variables. """
        f = np.zeros_like(Y)

        f[0] = Y[1]
        f[1] = -q*Y[1] - np.sin(Y[0]) + b*np.cos(omega_d*t)

        return f

def ddp_integrate(theta0, q=0.5, omega_d=2/3, b=0.9, dt=0.05, tmax=200):
    """integrate the damped driven pendulum systemm using the VODE method."""

    r = integrate.solve_ivp(
        partial(rhs, q=q, omega_d=omega_d, b=b),
        (0.0, tmax),
        [theta0, 0],
        method="RK45",
        dense_output=True,
    )

    ts = np.arange(0.0, tmax, dt)
    Xs = r.sol(ts)
    Xs[0] = restrict_theta(Xs[0])
    return ts, Xs

In [ ]:
%matplotlib inline
bs = np.arange(0.9, 1.4, 0.05)
theta0 = 0.5
tmax = 200

f, ax = plt.subplots(len(bs), 1, figsize=(10, 10))

for i, b in enumerate(bs):
    ts, Xs = ddp_integrate(theta0, b=b, tmax=tmax)
    ax[i].plot(ts, Xs[0])
    ts, Xs = ddp_integrate(theta0 + 0.0001, b=b, tmax=tmax)
    ax[i].plot(ts, Xs[0], zorder=0)
    ax[i].text(tmax + 3, 0, f"b = {b:0.2f}")
    ax[i].set_xlim(0, tmax)
    ax[i].set_ylabel(r"$\theta$")
    if i < len(bs) - 1:
        ax[i].set_xticklabels([])

ax[-1].set_xlabel("t [s]")

## Q8: FFT of the chaotic pendulum

In Q6 we looked at ODEs and the chaotic pendulum, and were interested in writing a method to integrate the pendulum in time.

Here we want to examine its behavior in frequency space.  The code below will integrate the chaotic pendulum, while requesting that the solution be stored at points spaced with a fixed dt, which makes it suitable for taking the FFT.

In [ ]:
from functools import partial
from scipy.integrate import solve_ivp

def rhs(t, Y, q, omega_d, b):
    """ damped driven pendulum system derivatives.  Here, Y = (theta, omega) are
        the solution variables. """
    f = np.zeros_like(Y)

    f[0] = Y[1]
    f[1] = -q*Y[1] - np.sin(Y[0]) + b*np.cos(omega_d*t)

    return f

def restrict_theta(theta):
    """ convert theta to be restricted to lie between -pi and pi"""
    tnew = theta + np.pi
    tnew += -2.0*np.pi*np.floor(tnew/(2.0*np.pi))
    tnew -= np.pi
    return tnew

def int_pendulum(theta0, q, omega_d, b, tend, dt):
    """ integrate the pendulum and return solution with dt"""

    # points in time where we'll request the solution
    tpoints = np.arange(0.0, tend, dt)

    r = solve_ivp(partial(rhs, q=q, omega_d=omega_d, b=b),
                  [0.0, tend], [theta0, 0.0],
                  method='RK45', t_eval=tpoints)

    return r.t, r.y

The call below will give an undamped pendulum.  For a small amplitude, since we have $L = g$ in our pendulum, the period is simply $T = 2\pi$, and the frequency is $\nu_k = 1/(2\pi)$.  We plot things in terms of angular frequency, $\omega_k = 2\pi \nu_k$, so all the power will be at $\omega_k = 1$.

In [ ]:
t, y = int_pendulum(np.radians(10), 0.0, 0.6666, 0.0, 200.0, 0.1)

Your task is to complete the power spectrum routine below to calculate the FFT of theta and plot it.  Experiment with the damping and driving parameters to see the complexity of the pendulum in frequency space when it becomes chaotic.  For reference, here's a plot of the solution theta

In [ ]:
plt.plot(t, restrict_theta(y[0,:]))
plt.xlabel("t")
plt.ylabel(r"$\theta$")

In [ ]:
def power_spectrum(t, theta0):
    """Return the power spectrum of theta wtih the frequency component in terms of omega."""
    theta = restrict_theta(theta0)
    freqs = np.linspace(0, 1/(2*(t[1]-t[0])), int(len(t)/2)+1)
    omega = 2 * np.pi * freqs
    pwr = np.abs(rfft(theta)) ** 2
    return omega, pwr

In [ ]:
%matplotlib notebook
from matplotlib.widgets import Button, Slider

init_q = 0.
init_b = 0.

fig, ax = plt.subplots(num="DDpendulum power spectrum")
t, y = int_pendulum(np.radians(10), init_q, 0.6666, init_b, 200.0, 0.1)
(line,) = ax.plot(*power_spectrum(t, y[0]), lw=2)
ax.set_xlabel("$\omega$ [rad/s]")
ax.set_ylabel("PSD [1/Hz]")
ax.set_yscale("log")
ax.set_ylim(1e-2, 1e7)
ax.grid()

fig.subplots_adjust(left=0.25, bottom=0.25)

axq = fig.add_axes([0.25, 0.9, 0.65, 0.03])
q_slider = Slider(
    ax=axq,
    label="q (damping)",
    valmin=0.0,
    valmax=5,
    valinit=init_q,
)

axb = fig.add_axes([0.25, 0.95, 0.65, 0.03])
b_slider = Slider(
    ax=axb,
    label="b (forcing amplitude)",
    valmin=0,
    valmax=5,
    valinit=init_b,
)


def update(val):
    t, y = int_pendulum(np.radians(10), q_slider.val, 0.6666, b_slider.val, 200.0, 0.1)
    line.set_ydata(power_spectrum(t, y[0])[1])
    fig.canvas.draw_idle()


b_slider.on_changed(update)
q_slider.on_changed(update)

resetax = fig.add_axes([0.8, 0.1, 0.1, 0.04])
button = Button(resetax, "Reset", hovercolor="0.975")


def reset(event):
    b_slider.reset()
    q_slider.reset()

button.on_clicked(reset)
plt.show()